In [ ]:
import os
import sys
import yt_dlp

In [ ]:
def mostrar_progreso(d):
    """Muestra el progreso de la descarga"""
    if d['status'] == 'downloading':
        porcentaje = d.get('_percent_str', '0%')
        velocidad = d.get('_speed_str', 'N/A')
        tiempo_restante = d.get('_eta_str', 'N/A')
        print(f"\rDescargando: {porcentaje} | Velocidad: {velocidad} | Tiempo restante: {tiempo_restante}", end='')
    elif d['status'] == 'finished':
        print("\nDescarga completada. Procesando archivo...")


def descargar_video(url, carpeta_destino="downloads"):
    """
    Descarga un video de YouTube en la más alta resolución disponible usando yt-dlp.
    
    Args:
        url: URL del video de YouTube
        carpeta_destino: Carpeta donde se guardará el video (por defecto 'descargas')
    
    Returns:
        La ruta completa del archivo descargado
    """
    try:

        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
        opciones = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
            'outtmpl': os.path.join(carpeta_destino, '%(title)s.%(ext)s'),
            'noplaylist': True,
            'progress_hooks': [mostrar_progreso],
            'quiet': False,
            'no_warnings': False,
            'ignoreerrors': False,
            'geo_bypass': True,
        }
        
        print(f"Conectando a YouTube para obtener información del video: {url}")
        

        with yt_dlp.YoutubeDL(opciones) as ydl:
            info = ydl.extract_info(url, download=False)
            
            if info:
                print(f"\nTítulo: {info.get('title', 'Desconocido')}")
                print(f"Duración: {info.get('duration', 'Desconocido')} segundos")
                print(f"Resolución máxima disponible: {info.get('resolution', 'Desconocido')}")
                
                print("\nIniciando descarga...")
                ydl.download([url])
                
                nombre_archivo = ydl.prepare_filename(info)
                print(f"\n¡Descarga completada! Archivo guardado en: {nombre_archivo}")
                return nombre_archivo
            else:
                print("No se pudo obtener información del video.")
                return None
    
    except Exception as e:
        print(f"\nError al descargar el video: {str(e)}")
        return None

In [ ]:
url = "https://www.youtube.com/watch?asdasdasd"
descargar_video(url)

In [ ]:
import subprocess
import os


def extract_audio_to_wav(
    input_path: str, 
    output_dir: str = "./"
) -> str:
    if not os.path.isfile(input_path):
        raise FileNotFoundError(f"[FFmpeg] Input file not found: {input_path}")

    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    wav_path = os.path.join(output_dir, f"{base_name}.wav")

    cmd = [
        "ffmpeg",
        "-y",
        "-i",
        input_path,
        "-vn",
        "-acodec",
        "pcm_s16le",
        "-ar",
        "16000",
        "-ac",
        "1",
        wav_path,
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return wav_path

In [ ]:
video_path = "downloads/<replace_mp4_filename>"
output_dir = "downloads"
wav_path = extract_audio_to_wav(video_path, output_dir)
wav_path